In [1]:
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = TapasTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-large-finetuned-wtq")

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--google--tapas-large-finetuned-wtq. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
# Example table in Pandas DataFrame
data = {
    "Year": ["2018", "2019", "2020", "2021"],
    "Sales": ["1000", "1500", "2000", "2500"]
}
table = pd.DataFrame.from_dict(data)

In [4]:
# Define your question
queries = ["What was the sales in 2019?", "What is the total sales?"]

# Tokenize inputs
inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")

# Pass the inputs through the model
outputs = model(**inputs)

# Get the predicted answer
logits = outputs.logits.detach().cpu().numpy()

# To interpret the logits, you can refer to the Hugging Face TAPAS documentation or use the tokenized outputs

c:\Python312\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
c:\Python312\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


In [13]:
# Convert logits to answer (example for cell selection)
predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(
    inputs, outputs.logits.detach()
)

# Print out the selected cells
for answer_set in predicted_answer_coordinates:
    # Check if answer_set is a list of lists
    if isinstance(answer_set, list) and all(isinstance(inner_list, list) for inner_list in answer_set):
        for coordinates in answer_set:  # Iterate over the inner lists
            # Check if each coordinates list contains tuples
            if all(isinstance(coord, tuple) for coord in coordinates):
                # Handle the inner list of (row, column) tuples
                cells = [table.iat[row, column] for row, column in coordinates]
                print("Answer (specific cells):", cells)
            else:
                print("Unexpected format for coordinates in answer set:", coordinates)
    else:
        print("Unexpected format for answer set:", answer_set)


Answer (specific cells): ['1500']
Answer (specific cells): ['1000', '1500', '2000', '2500']
